# Ansys CPFE UMAT Example
Material: cp-Ti (HCP)

In [1]:
!   Commands inserted into this file will be executed just prior to the ANSYS SOLVE command.
!   These commands may supersede command settings set by Workbench.

!   Active UNIT system in Workbench when this object was created:  Metric (um, kg, uN, s, V, mA)
!   NOTE:  Any data that requires units (such as mass) is assumed to be in the consistent solver unit system.
!                See Solving Units in the help system for more information.
/prep7
! Begin writing variables


CTE = 1e-05 ! Coefficient of thermal expansion in 1/K
EUL1 = 159 ! Euler angle 1 in degrees
EUL2 = 65  ! Euler angle 2 in degrees
EUL3 = 60  ! Euler angle 3 in degrees

! Begin writing grain materials
*SET,matid,1
TB,ELAS,matid,,9,
TBDATA,1,44714.0,44714.0,112368.0,12950.0,31600.0,31600.0,
TBDATA,7,0.7264,0.3062,0.3062
TB,CTE,matid,
TBDATA,1,CTE,CTE,CTE
TB,PLAS,matid
TB,XTAL,matid,,3,ORIE
TBDATA,1,EUL1,EUL2,EUL3
TB,XTAL,matid,,1,NSLFAM
TBDATA,1,5
TB,XTAL,matid,,1,FORM
TBDATA,1,1
TB,XTAL,matid,,10,XPARAM
TBDATA,1,2,0,45,30,1,1,
TBDATA,7,1,1,1,1
TB,XTAL,matid,1,18,HARD
TBTEMP, 298.0,
TBDATA,1,12,8.0,80.0,24.0,80.0,18.0,
TBDATA,7,18.0,180.0,48.0,180.0,12.0,12.0,
TBDATA,13,12.0,12.0,12.0,1.0,0.1,1.0
TB,XTAL,matid,1,3,FLHCP
TBTEMP, 298.0,
TBDATA,1,0.001,6.25,1.587
CMSEL,S,SX
EMODIF,ALL,MAT,matid
ALLSEL,ALL

allsel,all
TREF, 300.0
TUNIF, 300.0
/solu

SyntaxError: invalid syntax (2271305323.py, line 12)